In [1]:
import torch
if torch.cuda.is_available():
    print("✅ CUDA доступна")
    print("Устройство:", torch.cuda.get_device_name(0))
    print("Драйвер:", torch.version.cuda)
else:
    print("❌ CUDA НЕ доступна")


✅ CUDA доступна
Устройство: NVIDIA GeForce GTX 1070
Драйвер: 11.8


In [2]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")
model.train(data="dataset/data.yaml", epochs=50, imgsz=640, device=0)  # 0 — это GPU

100%|██████████| 5.35M/5.35M [00:00<00:00, 9.39MB/s]


New https://pypi.org/project/ultralytics/8.3.162 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.161  Python-3.13.5 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, 

train: Scanning D:\dev\Backpack_Battles_Helper_overlay\dataset\labels\train... 8000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8000/8000 [02:49<00:00, 47.07it/s]


train: New cache created: D:\dev\Backpack_Battles_Helper_overlay\dataset\labels\train.cache
val: Fast image access  (ping: 16.435.9 ms, read: 7.74.5 MB/s, size: 888.0 KB)


val: Scanning D:\dev\Backpack_Battles_Helper_overlay\dataset\labels\val... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:45<00:00, 44.28it/s]


val: New cache created: D:\dev\Backpack_Battles_Helper_overlay\dataset\labels\val.cache
Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=2.4e-05, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/500 [00:00<?, ?it/s]

WARNING CUDA OutOfMemoryError in TaskAlignedAssigner, using CPU


: 

Predict по фото

In [ ]:
from ultralytics import YOLO
import cv2

# Загрузка модели
model = YOLO('runs/detect/train12/weights/best.pt')

# Предсказание
results = model.predict(source='0.png', save=True, conf=0.1)

# Вывод результатов (bbox, классы)
for result in results:
    print(result.boxes)  # bounding boxes
    print(result.names)  # имена классов



image 1/1 d:\dev\Backpack_Battles_Helper\0.png: 512x640 (no detections), 14.1ms
Speed: 3.1ms preprocess, 14.1ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs\detect\predict15
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (1120, 1440)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))
{0: 'Lucky Piggy', 1: 'Heart of Darkness', 2: 'Flawed Ruby', 3: 'Rainbow Potion', 4: 'Twine', 5: 'Sapphire Whelp', 6: 'Gloves of Haste', 7: 'Stone Armor', 8: 'Mage Hat', 9: 'Armored Power Puppy', 10: 'Eggscalibur', 11: 'Puzzlebag of Endurance', 12: 'Rat Chef', 13: 'Ice Armor', 14: 'Shiny Mantle', 15: 'Hammer', 16: 'Strong Mana Potion', 17: 'Vi

realtime prediction

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import pygetwindow as gw
import pyautogui
import time

# Загрузка моделиD:\dev\Backpack_Battles_Helper\runs\detect\train10\weights\best.pt
model = YOLO("runs/detect/train12/weights/best.pt")

WINDOW_TITLE = "Backpack Battles"

# Смещение и размер рюкзака внутри окна
BAG_REGION_OFFSET = (30, 40, 760, 600)  # (offset_x, offset_y, width, height)

def get_window_rect(title_contains):
    for window in gw.getWindowsWithTitle(title_contains):
        if window.width > 0 and window.height > 0:
            return window.left, window.top, window.width, window.height
    raise RuntimeError(f"Окно с '{title_contains}' не найдено или свернуто.")

while True:
    try:
        # Получаем координаты окна
        win_left, win_top, win_width, win_height = get_window_rect(WINDOW_TITLE)

        # Координаты зоны рюкзака
        offset_x, offset_y, region_w, region_h = BAG_REGION_OFFSET
        abs_left = win_left + offset_x
        abs_top = win_top + offset_y

        # Захват изображения зоны рюкзака
        screenshot = pyautogui.screenshot(region=(abs_left, abs_top, region_w, region_h))
        frame = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

        # YOLOv8 предсказание
        results = model.predict(source=frame, conf=0.25, imgsz=640, verbose=False)[0]
        annotated_frame = results.plot()

        # Нарисуем зелёную рамку на копии изображения
        frame_with_box = annotated_frame.copy()
        cv2.rectangle(
            frame_with_box,
            (0, 0),
            (region_w - 1, region_h - 1),
            (0, 255, 0),
            thickness=3
        )

        # Отображение окна
        cv2.imshow("YOLOv8 - Backpack Region", frame_with_box)

        # Выход по Q
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    except Exception as e:
        print(f"[Ошибка] {e}")
        time.sleep(1)

cv2.destroyAllWindows()
